# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [7]:
# Import libraries
import numpy as np
import pandas as pd
import re

In [3]:
# Importing the dataset
pokemon = pd.read_csv('Pokemon.csv')

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [6]:
# your code here
pokemon['Sum']= pokemon['HP']+pokemon['Attack']+pokemon['Defense']+pokemon['Speed']+pokemon['Sp. Atk']+pokemon['Sp. Def']

pokemon['Confirm_Total']= pokemon['Total'] == pokemon['Sum']

pokemon['Confirm_Total'].unique()

print(f'''Total True, {pokemon['Confirm_Total'].eq(True).sum()}''')
print(f'''Total False, {pokemon['Confirm_Total'].eq(False).sum()}''')



Total True, 800
Total False, 0


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [18]:
# your code here


types = list(pokemon["Type 1"].unique())
    
#len for type 1
p1 = []
for i in range(len(pokemon)):
    p1.append(pokemon.iloc[i]["Type 1"])

#len for type 2
p2 = []
for i in range(len(pokemon)):
    p2.append(pokemon.iloc[i]["Type 2"])

#joining type 1 and 2 
p_type = []    
p_type = [[type1, type2] for type1, type2 in zip(p1, p2)]


table = []

for i in p_type:
    for item in types:
        if item in i:
            table.append(1)
        else:
            table.append(0)
            
table = [table[i:i+len(types)] for i in range(0,len(table),len(types))]

#print(table)

poke_types = pd.DataFrame(table, columns = types)

poke_types.head()



,Grass,Fire,Water,Bug,Normal,Poison,Electric,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Ice,Dragon,Dark,Steel,Flying
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [22]:
# your code here
poke_types["Total"] = pokemon["Total"]

corr = poke_types.corr()

corr1 = corr.sort_values(by=['Total'], ascending=False)

corr1.head(5)

# The pokemon that is most likely to have the highest Total is Dragon

,Grass,Fire,Water,Bug,Normal,Poison,Electric,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Ice,Dragon,Dark,Steel,Flying,Total
Total,-0.052592,0.078726,-0.021665,-0.145781,-0.105331,-0.090441,0.020971,0.015060,-0.036698,0.077786,0.124688,0.032731,0.003641,0.060248,0.229705,0.056154,0.109703,0.059383,1.000000
Dragon,-0.078818,-0.038069,-0.083285,-0.081200,-0.098702,-0.055525,-0.024000,0.052430,-0.035541,-0.068775,-0.026557,-0.032360,-0.019410,0.015173,1.000000,-0.003963,-0.044417,0.026237,0.229705
Psychic,-0.094009,-0.061243,-0.099639,-0.111968,-0.112381,-0.103195,-0.091928,-0.079079,0.027227,0.000596,1.000000,-0.069031,-0.070947,-0.042311,-0.026557,-0.044327,0.024540,-0.063872,0.124688
Steel,-0.061537,-0.056108,-0.096131,0.047178,-0.097645,-0.074036,-0.001346,-0.020771,0.013155,-0.005161,0.024540,0.049202,0.026479,-0.057042,-0.044417,-0.023978,1.000000,-0.081400,0.109703
Fire,-0.108248,1.000000,-0.114850,-0.060536,-0.085097,-0.085471,-0.057104,-0.022621,-0.067651,0.051130,-0.061243,-0.064677,-0.013459,-0.065851,-0.038069,-0.020368,-0.056108,-0.028855,0.078726


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [21]:
# your code here
